# Eat Safe, Love

## Notebook Set Up

In [272]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [273]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [274]:
# assign the uk_food database to a variable name
db = mongo['ukFood_db']

In [275]:
# review the collections in our database
collections = db.list_collection_names()
print(collections)

['establishments']


In [276]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [277]:
from pprint import pprint
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene":20}
result = establishments.find(query)
# Use count_documents to display the number of documents in the result
result_count = result.count()
print("Number of establishments with a hygiene score of 20:", result_count)
# Display the first document in the results using pprint
first_doc = result.next()
pprint(first_doc)

Number of establishments with a hygiene score of 20: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65cbef11f68591ad5c01d867'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'

/var/folders/q0/62b3m8qn6bvd3_bd3b18nw540000gn/T/ipykernel_564/4188841988.py:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  result_count = result.count()


In [278]:
import pandas as pd

# Convert the result cursor to a Pandas DataFrame
df = pd.DataFrame(list(result))

# Display the number of rows in the DataFrame
num_rows = len(df)
print("Number of rows in the DataFrame:", num_rows)

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 40
First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  65cbef11f68591ad5c01dbe5   612039                  0   
1  65cbef11f68591ad5c01deef   730933                  0   
2  65cbef11f68591ad5c01e0df   172735                  0   
3  65cbef11f68591ad5c01e0f0   172953                  0   
4  65cbef11f68591ad5c01ea8b   512854                  0   
5  65cbef11f68591ad5c01eca8  1537089                  0   
6  65cbef12f68591ad5c0201d6   155648                  0   
7  65cbef12f68591ad5c02061d  1012883                  0   
8  65cbef12f68591ad5c020e2d   644109                  0   
9  65cbef12f68591ad5c020eaf   267034                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                1970/FOOD                 Brenalwood   
1                1698/FOOD              Melrose Hotel   
2             PI/000023858              Seaford Pizza   
3             PI/000024532              Golden Palace 

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [279]:
from pprint import pprint

# Define the query to find establishments with London as the Local Authority and RatingValue >= 4
query = {"RatingValue": {"$gte": 4}}

# Find establishments with the specified criteria
result = establishments.find(query)

# Use count() to display the number of documents in the result
result_count = result.count()
print("Number of establishments with London as the Local Authority and RatingValue >= 4:", result_count)



Number of establishments with London as the Local Authority and RatingValue >= 4: 31257


/var/folders/q0/62b3m8qn6bvd3_bd3b18nw540000gn/T/ipykernel_564/4104019537.py:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  result_count = result.count()


In [280]:
import pandas as pd

# Convert the MongoDB query result to a Pandas DataFrame
df = pd.DataFrame(list(result))

# Display the number of rows in the DataFrame
num_rows = len(df)
print("Number of rows in the DataFrame:", num_rows)

# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in the DataFrame: 31257
                        _id   FHRSID  ChangesByServerID  \
0  65cbef11f68591ad5c01bd32   289353                  0   
1  65cbef11f68591ad5c01bd33   647177                  0   
2  65cbef11f68591ad5c01bd37   289352                  0   
3  65cbef11f68591ad5c01bd38   987206                  0   
4  65cbef11f68591ad5c01bd39   289560                  0   
5  65cbef11f68591ad5c01bd3a   344689                  0   
6  65cbef11f68591ad5c01bd3b   894592                  0   
7  65cbef11f68591ad5c01bd3c  1043695                  0   
8  65cbef11f68591ad5c01bd3e  1043701                  0   
9  65cbef11f68591ad5c01bd3f   805702                  0   

  LocalAuthorityBusinessID                         BusinessName  \
0             PI/000002468  St Marys COE (aided) Primary School   
1             PI/000041489                  Wear Bay Bowls Club   
2             PI/000002460                             The Ship   
3             PI/000075655                 

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [281]:
latitude = 51.5074
longitude = 0.1278
degree_search = 0.01

query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}

sort = [("scores.Hygiene", 1)]  # 1 = ascending order

result_select = establishments.find(query).sort(sort)

for doc in result_select:
    pprint(doc)



{'AddressLine1': 'Belvedere Road',
 'AddressLine2': 'London',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': 'The Ness Cafe',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4646.1445840265715,
 'FHRSID': 875250,
 'LocalAuthorityBusinessID': '14/00066/CP',
 'LocalAuthorityCode': '503',
 'LocalAuthorityEmailAddress': 'food.safety@bexley.gov.uk',
 'LocalAuthorityName': 'Bexley',
 'LocalAuthorityWebSite': 'http://www.bexley.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'SE2 9AQ',
 'RatingDate': '2018-06-05T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65cbef12f68591ad5c022c45'),
 'geocode': {'latitude': 51.506094, 'longitude': 0.133478},
 'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/875250',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T00:00:00',
          'i

In [282]:
# Convert result to Pandas DataFrame
import pandas as pd

# Execute the query and sort the results
result_select = establishments.find(query).sort(sort)

# Convert the cursor to a list of dictionaries
data = list(result_select)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 13
First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  65cbef12f68591ad5c022c45   875250                  0   
1  65cbef12f68591ad5c022c52  1064870                  0   
2  65cbef12f68591ad5c022dc5  1136999                  0   
3  65cbef12f68591ad5c022dc8  1215992                  0   
4  65cbef12f68591ad5c022e00  1392065                  0   
5  65cbef12f68591ad5c022e06  1409976                  0   
6  65cbef12f68591ad5c022f2f  1417255                  0   
7  65cbef12f68591ad5c022f68   329085                  0   
8  65cbef12f68591ad5c022fd2  1396471                  0   
9  65cbef12f68591ad5c022fd3  1385125                  0   

  LocalAuthorityBusinessID                                       BusinessName  \
0              14/00066/CP                                      The Ness Cafe   
1              18/00191/CP                                          Turo Turo   
2              19/00104/CP        Harri

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [283]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

pipeline = [
    {"$match": {"scores.Hygiene": 0}},  # Match establishments with a hygiene score of 0
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},  # Group by Local Authority and count documents
    {"$sort": {"count": -1}}  # Sort the results from highest to lowest count
]

# Execute the pipeline
result_cursor = establishments.aggregate(pipeline)
# Create a list variable to convert to datafram later
result_list = list(result_cursor)

# it gives me an error when I used the .count() function
# Print the number of documents in the result
result_count = 0
for result in result_cursor:
    result_count += 1

# Print the number of documents in the result
print("Number of documents in the result:", result_count)

# Re-execute the pipeline
result_cursor = establishments.aggregate(pipeline)

# Print the first 10 results
print("First 10 results:")
for i, result in enumerate(result_cursor):
    if i < 10:
        print(result)


        


Number of documents in the result: 0
First 10 results:
{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 882}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}


In [284]:
import pandas as pd

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result_list)

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 55
First 10 rows of the DataFrame:
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
